In [1]:
!nvidia-smi

Sun Nov 10 03:33:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/CNN

/content/drive/MyDrive/Colab Notebooks/CNN


In [4]:
import torch
import torchvision
from torchvision import transforms as transforms
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
import random

In [5]:
from torchvision import datasets

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

transform_test = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])


trainset = datasets.CIFAR10(
                    root='./data',
                    train=True,
                    download=True,
                    transform=transform_train)



testset = datasets.CIFAR10(
                    './data',
                    train=False,
                    download=True,
                    transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
trainloader = torch.utils.data.DataLoader(
                    trainset,
                    batch_size=128,
                    shuffle=True,
                    num_workers=8)

testloader = torch.utils.data.DataLoader(
                    testset,
                    batch_size=128,
                    shuffle=False,
                    num_workers=8)


In [7]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Model 1: ResNet18

In [8]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv = False, strides = 1):
        super().__init__()

        self.conv1 = nn.Conv2d(
            input_channels, num_channels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(
            num_channels, num_channels, kernel_size=3, padding=1)

        if use_1x1conv:
            self.conv3 = nn.Conv2d(
                input_channels, num_channels, kernel_size=1, stride = strides)
        else:
            self.conv3 = None

        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [9]:
b1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

def resnet_block(input_channels, num_channels, num_residuals, first_block = False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(
                Residual(input_channels, num_channels, use_1x1conv=True, strides=2))

        else:
            blk.append(Residual(num_channels, num_channels))
    return blk


b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))


ResNet18 = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))

In [10]:
X = torch.rand(size=(1, 3, 224, 224))
for layer in ResNet18:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


Model 2: VGG11

In [11]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(
            in_channels, out_channels, kernel_size = 3, padding=1 ))
        layers.append(nn.ReLU())
        in_channels = out_channels

    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [12]:
conv_arch = ((1,64), (1, 128), (2, 256), (2, 512), (2, 512))

def vgg(conv_arch):
    conv_blks = []
    in_channels = 3
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(
            num_convs, in_channels, out_channels ))
        in_channels = out_channels

    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        nn.Linear(out_channels *7*7, 4096), nn.ReLU(),
        nn.Dropout(0.5), nn.Linear(4096, 4096), nn.ReLU(),
        nn.Dropout(0.5), nn.Linear(4096, 10)
    )

VGG11Net = vgg(conv_arch)

In [13]:
X = torch.randn(size=(1, 3, 224, 224))
for blk in VGG11Net:
    X = blk(X)
    print(blk.__class__.__name__, 'output shape \t', X.shape)

Sequential output shape 	 torch.Size([1, 64, 112, 112])
Sequential output shape 	 torch.Size([1, 128, 56, 56])
Sequential output shape 	 torch.Size([1, 256, 28, 28])
Sequential output shape 	 torch.Size([1, 512, 14, 14])
Sequential output shape 	 torch.Size([1, 512, 7, 7])
Flatten output shape 	 torch.Size([1, 25088])
Linear output shape 	 torch.Size([1, 4096])
ReLU output shape 	 torch.Size([1, 4096])
Dropout output shape 	 torch.Size([1, 4096])
Linear output shape 	 torch.Size([1, 4096])
ReLU output shape 	 torch.Size([1, 4096])
Dropout output shape 	 torch.Size([1, 4096])
Linear output shape 	 torch.Size([1, 10])


Model 3: AlexNet

In [14]:
AlexNet = nn.Sequential(
    nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2), nn.Flatten(),
    nn.Linear(6400, 4096), nn.ReLU(), nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(p=0.5),
    nn.Linear(4096, 10))

In [15]:
X = torch.randn(1, 3, 224, 224)
for layer in AlexNet:
    X = layer(X)
    print(layer.__class__.__name__, 'Output shape:\t', X.shape)

Conv2d Output shape:	 torch.Size([1, 96, 54, 54])
ReLU Output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d Output shape:	 torch.Size([1, 96, 26, 26])
Conv2d Output shape:	 torch.Size([1, 256, 26, 26])
ReLU Output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d Output shape:	 torch.Size([1, 256, 12, 12])
Conv2d Output shape:	 torch.Size([1, 384, 12, 12])
ReLU Output shape:	 torch.Size([1, 384, 12, 12])
Conv2d Output shape:	 torch.Size([1, 384, 12, 12])
ReLU Output shape:	 torch.Size([1, 384, 12, 12])
Conv2d Output shape:	 torch.Size([1, 256, 12, 12])
ReLU Output shape:	 torch.Size([1, 256, 12, 12])
MaxPool2d Output shape:	 torch.Size([1, 256, 5, 5])
Flatten Output shape:	 torch.Size([1, 6400])
Linear Output shape:	 torch.Size([1, 4096])
ReLU Output shape:	 torch.Size([1, 4096])
Dropout Output shape:	 torch.Size([1, 4096])
Linear Output shape:	 torch.Size([1, 4096])
ReLU Output shape:	 torch.Size([1, 4096])
Dropout Output shape:	 torch.Size([1, 4096])
Linear Output shape:	 torch.Size([1,

Training

In [16]:
%load_ext tensorboard
writer = SummaryWriter()

In [17]:
from tqdm import tqdm

def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    loss_function = torch.nn.CrossEntropyLoss()
    mean_loss = torch.zeros(1).to(device)
    data_loader = tqdm(data_loader)
    for step, data in enumerate(data_loader):
        images, labels = data
        optimizer.zero_grad()
        pred = model(images.to(device))

        loss = loss_function(pred, labels.to(device))
        loss.backward()
        mean_loss = (mean_loss * step + loss.detach()) / (step + 1)  # update mean losses

        data_loader.desc = "[epoch {}] Train: mean loss {}".format(epoch, round(mean_loss.item(), 3))

        if not torch.isfinite(loss):
            print('WARNING: non-finite loss, ending training ', loss)

        optimizer.step()
    return mean_loss.item()

In [18]:
def prediction(model, data_iter, desc="prediction"):
    loss_function = torch.nn.CrossEntropyLoss()
    correct = 0
    total = 0
    mean_loss = torch.zeros(1).to(device)
    with torch.no_grad():
        for step, data in enumerate(data_iter):
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            loss = loss_function(outputs, labels.to(device))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            mean_loss = (mean_loss * step + loss.detach()) / (step + 1)
    acc = 100 * correct // total
    print(f'{desc}: Accuracy of the network: {acc} %')
    return acc, mean_loss.item()



def train(model, train_iter, test_iter, num_epochs, lr, device, name):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    num_batches = len(train_iter)
    model = model.to(device)

    for epoch in range(num_epochs):
        train_one_epoch(model, optimizer, train_iter, device, epoch)

        test_acc, test_loss = prediction(model, test_iter, 'test')
        train_acc, train_loss = prediction(model, train_iter, 'train')
        writer.add_scalars(f'{name}/loss', {"train": train_loss, "test": test_loss}, epoch)
        writer.add_scalars(f'{name}/accuracy', {"train": train_acc, "test": test_acc}, epoch)


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [20]:
train(AlexNet, trainloader, testloader, 10, 0.01, device, "AlexNet")

[epoch 0] Train: mean loss 1.929: 100%|██████████| 391/391 [00:26<00:00, 14.79it/s]


test: Accuracy of the network: 43 %
train: Accuracy of the network: 43 %


[epoch 1] Train: mean loss 1.355: 100%|██████████| 391/391 [00:25<00:00, 15.50it/s]


test: Accuracy of the network: 58 %
train: Accuracy of the network: 58 %


[epoch 2] Train: mean loss 1.048: 100%|██████████| 391/391 [00:25<00:00, 15.50it/s]


...

test: Accuracy of the network: 80 %
train: Accuracy of the network: 86 %


[epoch 8] Train: mean loss 0.424: 100%|██████████| 391/391 [00:24<00:00, 15.66it/s]


test: Accuracy of the network: 81 %
train: Accuracy of the network: 87 %


[epoch 9] Train: mean loss 0.382: 100%|██████████| 391/391 [00:24<00:00, 15.82it/s]


test: Accuracy of the network: 82 %
train: Accuracy of the network: 89 %


In [21]:
train(VGG11Net, trainloader, testloader, 10, 0.01, device, "VGG11Net")

[epoch 0] Train: mean loss 2.303: 100%|██████████| 391/391 [01:01<00:00,  6.35it/s]


test: Accuracy of the network: 10 %
train: Accuracy of the network: 10 %


[epoch 1] Train: mean loss 2.073: 100%|██████████| 391/391 [01:01<00:00,  6.33it/s]


test: Accuracy of the network: 34 %
train: Accuracy of the network: 33 %


[epoch 2] Train: mean loss 1.648: 100%|██████████| 391/391 [01:01<00:00,  6.35it/s]


...

test: Accuracy of the network: 72 %
train: Accuracy of the network: 79 %


[epoch 8] Train: mean loss 0.578: 100%|██████████| 391/391 [01:01<00:00,  6.32it/s]


test: Accuracy of the network: 74 %
train: Accuracy of the network: 82 %


[epoch 9] Train: mean loss 0.482: 100%|██████████| 391/391 [01:01<00:00,  6.32it/s]


test: Accuracy of the network: 75 %
train: Accuracy of the network: 87 %


In [22]:
train(ResNet18, trainloader, testloader, 10, 0.01, device, "ResNet18")

[epoch 0] Train: mean loss 1.338: 100%|██████████| 391/391 [00:37<00:00, 10.30it/s]


test: Accuracy of the network: 66 %
train: Accuracy of the network: 66 %


[epoch 1] Train: mean loss 0.776: 100%|██████████| 391/391 [00:37<00:00, 10.37it/s]


test: Accuracy of the network: 75 %
train: Accuracy of the network: 78 %


[epoch 2] Train: mean loss 0.567: 100%|██████████| 391/391 [00:38<00:00, 10.28it/s]


...

test: Accuracy of the network: 84 %
train: Accuracy of the network: 94 %


[epoch 8] Train: mean loss 0.163: 100%|██████████| 391/391 [00:38<00:00, 10.26it/s]


test: Accuracy of the network: 85 %
train: Accuracy of the network: 95 %


[epoch 9] Train: mean loss 0.128: 100%|██████████| 391/391 [00:38<00:00, 10.23it/s]


test: Accuracy of the network: 86 %
train: Accuracy of the network: 96 %


Learning Rate Decay

In [23]:
def train_lr_decay(model, train_iter, test_iter, num_epochs, init_lr, device, name):
    num_batches = len(train_iter)
    model = model.to(device)

    for epoch in range(num_epochs):
      decay_lr = init_lr / (epoch * 5.0 + 1)
      optimizer = torch.optim.SGD(model.parameters(), lr=decay_lr, momentum=0.9)
      train_one_epoch(model, optimizer, train_iter, device, epoch)

      test_acc, test_loss = prediction(model, test_iter, 'test')
      train_acc, train_loss = prediction(model, train_iter, 'train')
      writer.add_scalars(f'{name}/loss', {"train": train_loss, "test": test_loss}, epoch)
      writer.add_scalars(f'{name}/accuracy', {"train": train_acc, "test": test_acc}, epoch)

In [24]:
train_lr_decay(ResNet18, trainloader, testloader, 10, 0.1, device, "ResNet18_lr_decay")

[epoch 0] Train: mean loss 0.94: 100%|██████████| 391/391 [00:38<00:00, 10.26it/s]


test: Accuracy of the network: 77 %
train: Accuracy of the network: 79 %


[epoch 1] Train: mean loss 0.431: 100%|██████████| 391/391 [00:38<00:00, 10.29it/s]


test: Accuracy of the network: 83 %
train: Accuracy of the network: 87 %


[epoch 2] Train: mean loss 0.331: 100%|██████████| 391/391 [00:37<00:00, 10.29it/s]


...

test: Accuracy of the network: 86 %
train: Accuracy of the network: 94 %


[epoch 8] Train: mean loss 0.16: 100%|██████████| 391/391 [00:38<00:00, 10.27it/s]


test: Accuracy of the network: 86 %
train: Accuracy of the network: 95 %


[epoch 9] Train: mean loss 0.145: 100%|██████████| 391/391 [00:38<00:00, 10.22it/s]


test: Accuracy of the network: 86 %
train: Accuracy of the network: 95 %


Fine-Tuning

In [27]:
finetune_resnet18 = torchvision.models.resnet18(pretrained = True)
finetune_resnet18.fc = nn.Linear(finetune_resnet18.fc.in_features, 10)
nn.init.xavier_uniform_(finetune_resnet18.fc.weight)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 190MB/s]


Parameter containing:
tensor([[ 0.0427,  0.0838,  0.0309,  ...,  0.0025, -0.0181, -0.0250],
        [ 0.0901, -0.0015, -0.0887,  ..., -0.0802, -0.0355, -0.1047],
        [ 0.0463,  0.0037,  0.0928,  ..., -0.0952, -0.0345,  0.0694],
        ...,
        [-0.0972, -0.0473, -0.0706,  ..., -0.0212,  0.0948,  0.0081],
        [ 0.0388, -0.0742,  0.0520,  ..., -0.0227, -0.0164, -0.0566],
        [-0.0971, -0.0120, -0.0932,  ..., -0.0089,  0.0280,  0.0141]],
       requires_grad=True)

In [28]:
train(finetune_resnet18, trainloader, testloader, 10, 0.001, device,"ResNet18_fine_tune")

[epoch 0] Train: mean loss 0.569: 100%|██████████| 391/391 [00:36<00:00, 10.66it/s]


test: Accuracy of the network: 90 %
train: Accuracy of the network: 91 %


[epoch 1] Train: mean loss 0.218: 100%|██████████| 391/391 [00:36<00:00, 10.68it/s]


test: Accuracy of the network: 92 %
train: Accuracy of the network: 94 %


[epoch 2] Train: mean loss 0.156: 100%|██████████| 391/391 [00:36<00:00, 10.74it/s]


...

test: Accuracy of the network: 94 %
train: Accuracy of the network: 99 %


[epoch 8] Train: mean loss 0.039: 100%|██████████| 391/391 [00:36<00:00, 10.59it/s]


test: Accuracy of the network: 94 %
train: Accuracy of the network: 99 %


[epoch 9] Train: mean loss 0.031: 100%|██████████| 391/391 [00:36<00:00, 10.62it/s]


test: Accuracy of the network: 94 %
train: Accuracy of the network: 99 %
